### Processed 3 bands a time, and collected multiple images, using various strategies to combine them:

-	Majority Voting: Assign each pixel the most frequently predicted class from different band combinations.  
-	Weighted Averaging: Give more importance to specific bands or results with higher confidence scores.  
-	Feature Fusion via CNN or MLP: Use a post-processing model to learn the optimal way to fuse predictions from different band combinations.  


In [2]:
import numpy as np
from PIL import Image
import cv2

# Load your four segmentation images (binary masks)
common_path = '/Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/'
paths = [common_path+'dct_012.png', common_path+'dct_024.png', common_path+'dct_034.png', common_path+'dct_123.png']

# Load images and convert them into binary arrays
masks = [np.array(Image.open(p).convert('L')) for p in paths]

# Threshold images to binary values (assuming 0 and 255)
binary_masks = [(mask > 127).astype(np.uint8) for mask in masks]

# Stack masks along a new dimension
stacked_masks = np.stack(binary_masks, axis=0)

In [4]:
# Perform majority voting
# -------------------------------------------------------------------------------------------------
vote_sum = np.sum(stacked_masks, axis=0)
majority_vote = (vote_sum >= 3).astype(np.uint8) * 255  # 3 or more votes out of 4
# Save the resulting image
cv2.imwrite('/Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/merged_majority_vote.png', majority_vote)

True

In [6]:
# Perform Average-based Fusion
# -------------------------------------------------------------------------------------------------
avg_mask = np.mean(stacked_masks, axis=0)
soft_vote = (avg_mask > 0.5).astype(np.uint8) * 255
cv2.imwrite('/Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/merged_soft_vote.png', soft_vote)


True

In [7]:
# Intersection (Logical AND)keeps pixels segmented by all models.
# -------------------------------------------------------------------------------------------------
intersection_mask = np.all(stacked_masks, axis=0).astype(np.uint8) * 255
cv2.imwrite('/Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/merged_intersection.png', intersection_mask)


True

In [8]:
# Union (Logical OR) Pixels segmented by any model.
# -------------------------------------------------------------------------------------------------
union_mask = np.any(stacked_masks, axis=0).astype(np.uint8) * 255
cv2.imwrite('/Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/merged_union.png', union_mask)


True

In [9]:

# Weighted Average
# -------------------------------------------------------------------------------------------------
# Define weights for each model (must sum up to 1.0 ideally)
weights = [0.4, 0.3, 0.2, 0.1]

# Compute weighted average
weighted_sum = np.zeros_like(binary_masks[0], dtype=np.float32)
for mask, weight in zip(binary_masks, weights):
    weighted_sum += mask * weight

# Threshold the weighted sum at 0.5 to obtain final segmentation
weighted_avg_result = (weighted_sum >= 0.5).astype(np.uint8) * 255
# Save result
cv2.imwrite('/Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/weighted_average_result.png', weighted_avg_result)


True

In [ ]:
# Feature fusion typically involves combining multiple segmentation predictions at the feature-level 
# (e.g., confidence maps or logits) rather than binary outputs. 
# If your outputs are binary masks, feature fusion would be limited. 
# For richer feature fusion, you'd ideally have probability maps or feature maps directly from the model outputs.
# Max Fusion: Take pixel-wise max of all segmentation maps.
max_fusion = np.max(np.stack(binary_masks, axis=0), axis=0)
max_fusion_result = (max_fusion > 0).astype(np.uint8) * 255
cv2.imwrite('max_feature_fusion.png', max_fusion_result)


# Max Fusion: Take pixel-wise max of all segmentation maps.
multiplicative_fusion = np.prod(np.stack(binary_masks, axis=0), axis=0)
multiplicative_result = (multiplicative_fusion > 0).astype(np.uint8) * 255
cv2.imwrite('multiplicative_fusion.png', multiplicative_result)

### Combine RGB to get the true color
![multispectral camera setup](camera_setup.png)


### Check band's spatial resolution

In [10]:
import rasterio

with rasterio.open("/Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/IMG_0200_1.tif") as src:
    print(f"Band: {src.name}")
    print(f"Resolution: {src.res}")   # (pixel width, pixel height)
    print(f"Shape: {src.width} x {src.height}")
    
with rasterio.open("/Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/IMG_0200_2.tif") as src:
    print(f"Band: {src.name}")
    print(f"Resolution: {src.res}")   # (pixel width, pixel height)
    print(f"Shape: {src.width} x {src.height}")
    
with rasterio.open("/Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/IMG_0200_3.tif") as src:
    print(f"Band: {src.name}")
    print(f"Resolution: {src.res}")   # (pixel width, pixel height)
    print(f"Shape: {src.width} x {src.height}")

Band: /Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/IMG_0200_1.tif
Resolution: (1.0, 1.0)
Shape: 1280 x 960
Band: /Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/IMG_0200_2.tif
Resolution: (1.0, 1.0)
Shape: 1280 x 960
Band: /Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/IMG_0200_3.tif
Resolution: (1.0, 1.0)
Shape: 1280 x 960


/opt/anaconda3/envs/hsi/lib/python3.10/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


### RGB bands are slightly offset. Need to align them.

In [ ]:
import cv2
import numpy as np

# -------- CONFIG --------
# red_path = 'Metashape_aligned/IMG_0200_3.tif'
# green_path = 'Metashape_aligned/IMG_0200_2.tif'
# blue_path = 'Metashape_aligned/IMG_0200_1.tif'
# nir_path = 'Metashape_aligned/IMG_0200_4.tif'
# red_edge_path = 'Metashape_aligned/IMG_0200_5.tif'
# output_path = 'Metashape_aligned/red_edge_aligned_rgb.png'

red_path = 'IMG_0200_3.tif'
green_path = 'IMG_0200_2.tif'
blue_path = 'IMG_0200_1.tif'
nir_path = 'IMG_0200_4.tif'
red_edge_path = 'IMG_0200_5.tif'
output_path = 'red_edge_aligned_rgb.png'
# ------------------------

# Load 16-bit images
red = cv2.imread(red_path, cv2.IMREAD_UNCHANGED)
green = cv2.imread(green_path, cv2.IMREAD_UNCHANGED)
blue = cv2.imread(blue_path, cv2.IMREAD_UNCHANGED)
nir = cv2.imread(nir_path, cv2.IMREAD_UNCHANGED)
red_edge = cv2.imread(red_edge_path, cv2.IMREAD_UNCHANGED)

# Alignment function
def align_images(base, to_align):
    warp_matrix = np.eye(2, 3, dtype=np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000, 1e-10)

    try:
        cc, warp_matrix = cv2.findTransformECC(
            cv2.normalize(base, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8),
            cv2.normalize(to_align, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8),
            warp_matrix,
            cv2.MOTION_TRANSLATION,
            criteria
        )
    except:
        print("Alignment failed, using original image")
        return to_align

    aligned = cv2.warpAffine(to_align, warp_matrix, (base.shape[1], base.shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    return aligned

# Align bands to red
red_aligned = align_images(red_edge, red)
green_aligned = align_images(red_edge, green)
blue_aligned = align_images(red_edge, blue)

# Normalize for PNG (convert 16-bit to 8-bit)
def normalize_to_uint8(img):
    norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    return norm.astype(np.uint8)

red_8 = normalize_to_uint8(red_aligned)
green_8 = normalize_to_uint8(green_aligned)
blue_8 = normalize_to_uint8(blue_aligned)

# Merge and save
rgb_8bit = cv2.merge([blue_8, green_8, red_8])
cv2.imwrite(output_path, rgb_8bit)

print(f"Saved aligned RGB image to: {output_path}")

Saved aligned RGB image to: red_edge_aligned_rgb.png


### Automated Feature-based alignment

In [9]:
import cv2
import numpy as np

def align_image(ref_path, src_path, detector=None):
    """
    Align the source image to the reference image using ORB feature matching
    and homography estimation.

    Parameters:
      ref_path (str): File path to the reference image.
      src_path (str): File path to the source image to be aligned.
      detector (cv2.Feature2D, optional): Feature detector (e.g., ORB). If None, a default ORB detector is created.

    Returns:
      aligned_src (np.ndarray): The aligned version of the source image.
    """
    # Load images in grayscale
    ref_img = cv2.imread(ref_path, cv2.IMREAD_GRAYSCALE)
    src_img = cv2.imread(src_path, cv2.IMREAD_GRAYSCALE)

    if ref_img is None:
        raise IOError(f"Reference image '{ref_path}' could not be loaded.")
    if src_img is None:
        raise IOError(f"Source image '{src_path}' could not be loaded.")

    # Use provided detector or create one.
    if detector is None:
        detector = cv2.ORB_create(500)

    # Detect keypoints and compute descriptors for both images.
    kp_ref, des_ref = detector.detectAndCompute(ref_img, None)
    kp_src, des_src = detector.detectAndCompute(src_img, None)

    # Create the brute-force matcher and match descriptors.
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des_ref, des_src)
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract the coordinates for the best matches.
    pts_ref = np.float32([kp_ref[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts_src = np.float32([kp_src[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Compute the homography matrix using RANSAC.
    H, status = cv2.findHomography(pts_src, pts_ref, cv2.RANSAC, 5.0)
    if H is None:
        raise ValueError(f"Homography could not be computed between '{ref_path}' and '{src_path}'.")

    # Warp the source image to align with the reference image.
    aligned_src = cv2.warpPerspective(src_img, H, (ref_img.shape[1], ref_img.shape[0]))

    return aligned_src

if __name__ == "__main__":
    # -------- CONFIG --------
    red_path = 'IMG_0200_3.tif'
    green_path = 'IMG_0200_2.tif'
    blue_path = 'IMG_0200_1.tif'
    nir_path = 'IMG_0200_4.tif'
    red_edge_path = 'IMG_0200_5.tif'
    output_path = 'red_edge_aligned_rgb.png'

    # Create a shared ORB detector (optional but ensures consistency between alignments).
    orb_detector = cv2.ORB_create(500)

    # Align each RGB band to the red_edge reference.
    aligned_red   = align_image(nir_path, red_path, detector=orb_detector)
    aligned_green = align_image(nir_path, green_path, detector=orb_detector)
    aligned_blue  = align_image(nir_path, blue_path, detector=orb_detector)

    # Merge channels to create an RGB composite.
    # Note: OpenCV uses channel order [Blue, Green, Red].
    rgb_composite = cv2.merge([aligned_blue, aligned_green, aligned_red])

    # Save the final composite image.
    out_rgb_filename = 'nir_feature_aligned_rgb_2.png'
    cv2.imwrite(out_rgb_filename, rgb_composite)
    print(f"Aligned RGB composite saved as '{out_rgb_filename}'.")

Aligned RGB composite saved as 'nir_feature_aligned_rgb_2.png'.


### Combine one image of 3 brands 

In [ ]:
import rasterio
import numpy as np
from PIL import Image

# Paths to your individual band files
red_path = 'IMG_0200_3.tif'
green_path = 'IMG_0200_2.tif'
blue_path = 'IMG_0200_1.tif'

# Read each band
with rasterio.open(red_path) as red_src:
    red = red_src.read(1)
    profile = red_src.profile  # To use for output if needed

with rasterio.open(green_path) as green_src:
    green = green_src.read(1)

with rasterio.open(blue_path) as blue_src:
    blue = blue_src.read(1)

# Stack into RGB
rgb = np.stack([red, green, blue], axis=-1)

# Optional: Normalize to 0–255 if not already in that range
rgb_normalized = (255 * (rgb / rgb.max())).astype(np.uint8)

# Save as PNG image
Image.fromarray(rgb_normalized).save('0200_rgb.png')

# OR — Save as a new multi-band GeoTIFF
# with rasterio.open('output_rgb.tif', 'w', **profile) as dst:
#     dst.write(rgb[:, :, 0], 1)  # R
#     dst.write(rgb[:, :, 1], 2)  # G
#     dst.write(rgb[:, :, 2], 3)  # B

### combine all the images of 3 bands.

In [ ]:
import os
import glob
import numpy as np
import rasterio
from PIL import Image

# Set the folder path where the .tif files are stored
folder_path = "/Users/seaqueue/Downloads/blueberry/batch1/"  # change this to your folder

# Find all .tif files in the folder
file_list = glob.glob(os.path.join(folder_path, "IMG_*_*.tif"))

# Dictionary to store files by location
locations = {}

for filepath in file_list:
    filename = os.path.basename(filepath)
    # Expecting a pattern like "IMG_0200_1.tif"
    try:
        # Split filename: "IMG", "0200", "1.tif"
        parts = filename.split("_")
        location = "_".join(parts[:2])  # e.g., "IMG_0200"
        band_str = parts[-1].split('.')[0]  # e.g., "1"
    except IndexError:
        print(f"Skipping file with unexpected format: {filename}")
        continue

    if location not in locations:
        locations[location] = {}
    locations[location][band_str] = filepath

# Process each location
for loc, bands in locations.items():
    # print(loc, bands)
    # Check if bands 1, 2, and 3 exist
    if all(b in bands for b in ['1', '2', '3']):
        # Read each band using rasterio
        with rasterio.open(bands['1']) as src:
            blue = src.read(1)
        with rasterio.open(bands['2']) as src:
            green = src.read(1)
        with rasterio.open(bands['3']) as src:
            red = src.read(1)
        
        # Stack the bands into an (height, width, 3) array in RGB order
        # Note: We are mapping band 1 -> Blue, band 2 -> Green, band 3 -> Red.
        rgb_array = np.dstack((red, green, blue))
        
        rgb_normalized = (255 * (rgb_array / rgb_array.max())).astype(np.uint8)
        image = Image.fromarray(rgb_normalized)
        # image = Image.fromarray(rgb_array.astype(np.uint8))
        output_filename = os.path.join("/Users/seaqueue/Downloads/blueberry/batch1_rgb/", f"{loc}_rgb.png")
        image.save(output_filename)
        print(f"Saved RGB image for {loc} as {output_filename}")
    else:
        print(f"Missing required bands for location {loc}, skipping.")